<a href="https://colab.research.google.com/github/sergio-qp/AP_2_SQ_AG/blob/master/Proyect_2_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [127]:
import pandas as pd
from google.colab import files
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score, balanced_accuracy_score, confusion_matrix
import time
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler
from sklearn.model_selection import GridSearchCV





In [42]:

uploaded=files.upload()
file_name = list(uploaded.keys())[0]
data = pd.read_csv(file_name)

Saving attrition_availabledata_02.csv to attrition_availabledata_02 (3).csv


In [73]:
print(data.head())

         hrs  absences  JobInvolvement  PerformanceRating  \
0  10.060048       6.0             3.0                4.0   
1   9.437671       2.0             2.0                3.0   
2   7.900932      20.0             3.0                4.0   
3   7.193853      19.0             4.0                3.0   
4   6.979201       8.0             3.0                3.0   

   EnvironmentSatisfaction  JobSatisfaction  WorkLifeBalance   Age  \
0                      2.0              4.0              1.0  31.0   
1                      3.0              4.0              3.0  33.0   
2                      3.0              4.0              3.0  35.0   
3                      4.0              2.0              3.0  28.0   
4                      2.0              4.0              2.0  31.0   

      BusinessTravel              Department  ...  Over18  PercentSalaryHike  \
0  Travel_Frequently  Research & Development  ...       Y               23.0   
1         Non-Travel  Research & Development  ...   

In [116]:

# Información general
num_rows = len(data)  # Número de filas
num_columns = len(data.columns)  # Número de columnas

# Tipos de columnas
column_types = data.dtypes  # Tipos de datos de las columnas
column_info = {}
for dtype in column_types.unique():
    column_info[str(dtype)] = sum(column_types == dtype)

# Identificar variables categóricas y numéricas
categorical_vars = [col for col in data.columns if data[col].dtype == 'object']
numerical_vars = [col for col in data.columns if data[col].dtype in ['int64', 'float64']]

# Cardinalidad de las variables categóricas
categorical_cardinality = {}
for col in categorical_vars:
    categorical_cardinality[col] = data[col].nunique()

# Valores faltantes

if data[col].isnull().sum().sum()>0:
    missing_values="There are missing values"

else:
  missing_values="There are not missing values"

# Columnas constantes
constant_columns = [col for col in data.columns if data[col].nunique() == 1]


# Columnas de ID
possible_id_columns = []
for col in data.columns:
    if data[col].nunique() == num_rows:
        possible_id_columns.append(col)

# Tipo de problema (regresión o clasificación)
target_variable = data.columns[-1]  # Última columna como objetivo
if data[target_variable].nunique() <= 10:
    target_type = 'classification'
else:
    target_type = 'regression'

# Distribución de clases (si es clasificación)
if target_type == 'classification':
    class_counts = data[target_variable].value_counts()  # Cuenta cada clase
    total_counts = len(data[target_variable])  # Total de filas
    class_distribution = class_counts / total_counts  # Proporción de cada clase

    # Verifica si el dataset está desbalanceado
    is_imbalanced = class_distribution.max() > 0.6  # Usa .max() con paréntesis
else:
    class_distribution = None
    is_imbalanced = None

# Resumen del EDA
eda_summary = {
    "Number of Rows": num_rows,
    "Number of Columns": num_columns,
    "Column Types": column_info,
    "Categorical Variables": categorical_vars,
    "Numerical Variables": numerical_vars,
    "Categorical Cardinality": categorical_cardinality,
    "Missing Values": missing_values,
    "Constant Columns": constant_columns,
    "Possible ID Columns": possible_id_columns,
    "Problem Type": target_type,
    "Class Distribution (if classification)": class_distribution,
    "Is Imbalanced": is_imbalanced,
}


for key, value in eda_summary.items():
    print(f"{key}: {value}")



Number of Rows: 2940
Number of Columns: 28
Column Types: {'float64': 21, 'object': 7}
Categorical Variables: ['BusinessTravel', 'Department', 'EducationField', 'Gender', 'JobRole', 'MaritalStatus', 'Attrition']
Numerical Variables: ['hrs', 'absences', 'JobInvolvement', 'PerformanceRating', 'EnvironmentSatisfaction', 'JobSatisfaction', 'WorkLifeBalance', 'Age', 'DistanceFromHome', 'Education', 'EmployeeID', 'JobLevel', 'MonthlyIncome', 'NumCompaniesWorked', 'PercentSalaryHike', 'StockOptionLevel', 'TotalWorkingYears', 'TrainingTimesLastYear', 'YearsAtCompany', 'YearsSinceLastPromotion', 'YearsWithCurrManager']
Categorical Cardinality: {'BusinessTravel': 3, 'Department': 3, 'EducationField': 6, 'Gender': 2, 'JobRole': 9, 'MaritalStatus': 3, 'Attrition': 2}
Missing Values: There are not missing values
Constant Columns: []
Possible ID Columns: ['EmployeeID']
Problem Type: classification
Class Distribution (if classification): Attrition
No     0.838776
Yes    0.161224
Name: count, dtype: fl

The dataset contains 2940 rows and 31 columns. Of these 31 variables, 23 are numerical variables, and 8 are categorical variables. These categorical variables include: BusinessTravel, Department, or Gender. They have different values; for example, Gender has two unique values: "Female" and "Male." On the other hand, there are other categorical variables, such as JobRole, which have 9 unique values, indicating different roles like Sales Executive or Manager.

Before performing any statistical analysis, it is crucial to check if the dataset contains missing values, as they can cause errors. As observed, the dataset does not have any missing values. Similarly, it is essential to verify if there are variables that have the same value across all observations, as these variables could be removed. In this dataset, Over18, EmployeeCount, and StandardHours are constant variables, which do not provide any useful information and should be eliminated.

To determine whether we have a classification or regression problem, it is necessary to identify our target variable. The target variable is Attrition, whose unique values are "Yes" and "No." Therefore, as it is a categorical variable, this is a classification problem.

The target variable presents significant imbalance, with 83% of the observations corresponding to employees who do not leave ("No") and 16% belonging to the "Yes" category.

In [134]:
# Remove constant columns from the DataFrame
data = data.drop(columns=constant_columns)

# Display the removed columns
print(f"Constant Columns Removed: {constant_columns}")

# Update and display the new number of rows and columns
num_rows = len(data)
num_columns = len(data.columns)

print(f"Updated DataFrame: {num_rows} rows and {num_columns} columns.")


Constant Columns Removed: []
Updated DataFrame: 2940 rows and 28 columns.


In [132]:
#SET UP
# Separate features (X) and target variable (y)
X = data.drop(columns=['Attrition'])  # Drop target variable
y = data['Attrition'].replace({'No': 0, 'Yes': 1})
# Division de los datos
# Split the dataset into training and testing sets (80/20 split)
seed = 123456  # Set a fixed seed for reproducibility
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed, stratify=y)



<ipython-input-132-125a3e86c9fa>:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  y = data['Attrition'].replace({'No': 0, 'Yes': 1})


The Holdout method involves splitting the dataset into training and test sets. In this case, 80% of the observations were assigned to the training set, and 20% to the test set.

Holdout was used because it simulates the model's effectiveness on completely new data. Additionally, since the target variable is imbalanced, it is essential to maintain the proportion of classes in both samples to ensure that one sample does not contain only one class of the target variable.

In [133]:
# Step 1: Identify categorical and numerical columns
# This separates the features into categorical and numerical types for preprocessing.
categorical_vars = X.select_dtypes(include=['object']).columns.tolist()  # List of categorical columns
numerical_vars = X.select_dtypes(include=['int64', 'float64']).columns.tolist()  # List of numerical columns

# Step 2: Create a preprocessor
# A ColumnTransformer is used to apply different transformations to categorical and numerical columns.
# - Categorical: OneHotEncoder to convert categories into binary columns.
# - Numerical: 'passthrough' means numerical columns are not transformed.
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_vars),  # Transform categorical variables
        ('num', 'passthrough', numerical_vars)  # Keep numerical variables as is
    ]
)

# Method 1: Dummy Classifier (Baseline)
# A simple classifier that always predicts the most frequent class. This is used as a baseline for comparison.
dummy_clf = DummyClassifier(strategy="most_frequent", random_state=42)  # Most frequent strategy
dummy_clf.fit(X_train, y_train)  # Train on the training data
dummy_preds = dummy_clf.predict(X_test)  # Predict on the test data
dummy_acc = accuracy_score(y_test, dummy_preds)  # Calculate accuracy
dummy_bal_acc = balanced_accuracy_score(y_test, dummy_preds)  # Calculate balanced accuracy

# Method 2: Decision Tree with Pipeline
# Combines the preprocessor and a Decision Tree model into a single pipeline for clean and efficient workflow.
tree_pipeline = Pipeline([
    ('preprocessor', preprocessor),  # Apply preprocessing to the data
    ('classifier', DecisionTreeClassifier(random_state=42))  # Train a Decision Tree classifier
])
tree_pipeline.fit(X_train, y_train)  # Train the pipeline
tree_preds = tree_pipeline.predict(X_test)  # Predict using the trained pipeline
tree_acc = accuracy_score(y_test, tree_preds)  # Calculate accuracy
tree_bal_acc = balanced_accuracy_score(y_test, tree_preds)  # Calculate balanced accuracy

# Method 3: KNN with StandardScaler
# Combines preprocessing, scaling (StandardScaler), and KNN model into a single pipeline.
knn_pipeline_std = Pipeline([
    ('preprocessor', preprocessor),  # Apply preprocessing
    ('scaler', StandardScaler()),  # Scale features to have mean=0 and std=1
    ('classifier', KNeighborsClassifier())  # Train a KNN classifier
])
knn_pipeline_std.fit(X_train, y_train)  # Train the pipeline
knn_std_preds = knn_pipeline_std.predict(X_test)  # Predict using the trained pipeline
knn_std_acc = accuracy_score(y_test, knn_std_preds)  # Calculate accuracy
knn_std_bal_acc = balanced_accuracy_score(y_test, knn_std_preds)  # Calculate balanced accuracy

# Method 4: KNN with MinMaxScaler
# Similar to the StandardScaler pipeline, but uses MinMaxScaler for scaling.
knn_pipeline_minmax = Pipeline([
    ('preprocessor', preprocessor),  # Apply preprocessing
    ('scaler', MinMaxScaler()),  # Scale features to range [0, 1]
    ('classifier', KNeighborsClassifier())  # Train a KNN classifier
])
knn_pipeline_minmax.fit(X_train, y_train)  # Train the pipeline
knn_minmax_preds = knn_pipeline_minmax.predict(X_test)  # Predict using the trained pipeline
knn_minmax_acc = accuracy_score(y_test, knn_minmax_preds)  # Calculate accuracy
knn_minmax_bal_acc = balanced_accuracy_score(y_test, knn_minmax_preds)  # Calculate balanced accuracy

# Step 5: Display results
# Print the performance metrics (Accuracy and Balanced Accuracy) for all models.
print("Results:")
print(f"Dummy Classifier - Accuracy: {dummy_acc:.2f}, Balanced Accuracy: {dummy_bal_acc:.2f}")
print(f"Decision Tree - Accuracy: {tree_acc:.2f}, Balanced Accuracy: {tree_bal_acc:.2f}")
print(f"KNN (StandardScaler) - Accuracy: {knn_std_acc:.2f}, Balanced Accuracy: {knn_std_bal_acc:.2f}")
print(f"KNN (MinMaxScaler) - Accuracy: {knn_minmax_acc:.2f}, Balanced Accuracy: {knn_minmax_bal_acc:.2f}")




Results:
Dummy Classifier - Accuracy: 0.84, Balanced Accuracy: 0.50
Decision Tree - Accuracy: 0.92, Balanced Accuracy: 0.84
KNN (StandardScaler) - Accuracy: 0.86, Balanced Accuracy: 0.66
KNN (MinMaxScaler) - Accuracy: 0.86, Balanced Accuracy: 0.67


For this section, the following procedures were carried out:

Identification of categorical and numerical columns: As performed in the EDA section, categorical and numerical variables were identified.
Transformation of categorical variables: Categorical variables were converted into binary variables using OneHotEncoder.
Dummy Classifier: This model predicts the most frequent class in the dataset.
Decision Tree and KNN: A Decision Tree model was implemented, as well as KNN, which compares distances between data points. For KNN, the scales of variables can significantly impact the model's performance. Therefore, two types of scaling methods were compared: StandardScaler and MinMaxScaler.
Evaluation of classification methods: The effectiveness of the various classification methods was calculated and compared.


In [128]:

# Paso 1: Identificar variables categóricas y numéricas
categorical_vars = X.select_dtypes(include=['object']).columns.tolist()
numerical_vars = X.select_dtypes(include=['int64', 'float64']).columns.tolist()

# Paso 2: Crear un preprocesador
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_vars),  # Codificar categóricas
        ('num', 'passthrough', numerical_vars)  # Mantener numéricas
    ]
)

# Paso 3: Pipeline para Decision Tree
tree_pipeline = Pipeline([
    ('preprocessor', preprocessor),  # Preprocesamiento
    ('classifier', DecisionTreeClassifier(random_state=42))  # Modelo
])

# Entrenar Decision Tree
tree_pipeline.fit(X_train, y_train)
tree_preds = tree_pipeline.predict(X_test)
tree_acc = accuracy_score(y_test, tree_preds)
print(f"Decision Tree Accuracy: {tree_acc:.2f}")

# Paso 4: Pipeline para KNN con StandardScaler
knn_pipeline_std = Pipeline([
    ('preprocessor', preprocessor),  # Preprocesamiento
    ('scaler', StandardScaler()),  # Escalado estándar
    ('classifier', KNeighborsClassifier())  # Modelo KNN
])

# Entrenar KNN con StandardScaler
knn_pipeline_std.fit(X_train, y_train)
knn_std_preds = knn_pipeline_std.predict(X_test)
knn_std_acc = accuracy_score(y_test, knn_std_preds)
print(f"KNN (StandardScaler) Accuracy: {knn_std_acc:.2f}")

# Paso 5: Pipeline para KNN con MinMaxScaler
knn_pipeline_minmax = Pipeline([
    ('preprocessor', preprocessor),  # Preprocesamiento
    ('scaler', MinMaxScaler()),  # Escalado Min-Max
    ('classifier', KNeighborsClassifier())  # Modelo KNN
])

# Entrenar KNN con MinMaxScaler
knn_pipeline_minmax.fit(X_train, y_train)
knn_minmax_preds = knn_pipeline_minmax.predict(X_test)
knn_minmax_acc = accuracy_score(y_test, knn_minmax_preds)
print(f"KNN (MinMaxScaler) Accuracy: {knn_minmax_acc:.2f}")




Decision Tree Accuracy: 0.92
KNN (StandardScaler) Accuracy: 0.86
KNN (MinMaxScaler) Accuracy: 0.86
Best Parameters for Decision Tree: {'classifier__max_depth': 13, 'classifier__min_samples_split': 2}
Best Cross-Validated Accuracy for Decision Tree: 0.87
Best Parameters for KNN (StandardScaler): {'classifier__n_neighbors': 1}
Best Cross-Validated Accuracy for KNN (StandardScaler): 0.90


In [ ]:
# Inner evaluation with 3-fold cross-validation
inner_cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=seed)